## Find the derivative $\frac{d f}{d x}$ of the function 
$$f(x) = x^3cos(x) + x^2sin(x)^2 + log(3x)$$ 

Packages used: 
<span style="color:yellow; background-color:green">Symbolics, Latexify</span>

In [1]:
using Symbolics, Latexify

### **Symbolic Expressions**

- `@variable x` ⇒ Declares a symbolic variable `x` for use in expressions.
- `Differential(x)(f)` ⇒ Represents the derivative of a function `f` with respect to the variable `x`.
- `expand_derivatives(df)` ⇒ Expands the derivatives in a symbolic expression, showing them in full form.
- `simplify()` ⇒ Simplifies a symbolic expression, reducing it to a more concise form.

In [2]:
# Create custom function to make differentiation convenient
function deriv(f::Symbolics.Num, x::Symbolics.Num)
    # expand_derivatives => show the result of differentiation
    df = Differential(x)(f)
    return simplify(expand_derivatives(df))
end

# Makes x a symbolic variable
@variables x

# just f
f = x^3*cos(x) + x^2*sin(x)^2 + log(3x)
display(f)

# Differential(x)(f) => show d/dx (f)
Differential(x)(f)

dfdx = deriv(f,x)
display(dfdx)

log(3x) + (x^3)*cos(x) + (x^2)*(sin(x)^2)

1 / x + 3(x^2)*cos(x) + (x^2)*sin(2x) + 2x*(sin(x)^2) - (x^3)*sin(x)

### **Evaluation Expression**

- `Dict(x => x0)` ⇒ A mapping that replaces the variable `x` with the value `x0`.
- `substitute(f, (Dict(x => x0)))` ⇒ Returns a `Num` type expression with the symbolic variable `x` replaced by the value `x0`.
- `value(...)` ⇒ Converts the symbolic expression into a `Float64` value after substitution.
- `get_variables()` ⇒ Extracts the symbolic variables from an expression (e.g., `f`).
- `first()` ⇒ Returns the first element of a collection (such as a list or array).

In [3]:
# Let f be a symbolic expression (i.e., function of @variables x)
function evaluate(f::Any, x0::Float64)
    vars = Symbolics.get_variables(f)
    return Symbolics.value(substitute(f, (Dict(first(vars) => x0))))
end

evaluate(dfdx, .5)

3.0384753223601586

### **Vector version of `deriv` and `evaluate2`**

HELPER FUNCTIONS
- `hcat(...)` ⇒ Horizontally concatenates values into a matrix or array.
- `Ref(...)` ⇒ Wraps a value in a reference to avoid broadcasting issues. Useful when applying the same dictionary to multiple expressions.
- `Dict(...)` ⇒ Creates a dictionary mapping symbolic variables to their numerical values.
- `zip(a, b)` ⇒ Pairs elements from two collections (like variables and values) into key-value tuples.
- `vcat(...)` ⇒ Vertically concatenates arrays — used to flatten nested arrays (e.g., from a list of vectors).
- `unique(...)` ⇒ Removes duplicates from a list, ensuring each variable only appears once.

In [37]:
function deriv(f::Vector{Symbolics.Num}, x::Symbolics.Num)
    df = []
    for component in f
        df_comp = Differential(x)(component)
        push!(df, simplify(expand_derivatives(df_comp)))
    end
    return df
end

function evaluate(f::Vector{Num}, x0::Vector{Float64})
    vars = unique(vcat(Symbolics.get_variables.(f)...))
    subs = Ref(Dict(zip(vars, x0)))
    return (Symbolics.value.(substitute.(f, subs)))
end

# function evaluate(f::Vector{Num}, x0::Vector{Float64})
#     vars = Symbolics.get_variables(f[1])
#     var = first(vars)
#     return hcat([(Symbolics.value.(substitute.(f, Ref(var => x_val)))) for x_val in x0]...)
# end


evaluate (generic function with 2 methods)

In [38]:
@variables x1 x2
fc = [x1 * (x2)^2; x2*( sin(x1) )^3 + 11; 4*(x1)^2 + ( x2 +3 )^2]
x0 = [0.0; 0.0]

dfcdx1 = deriv.(fc, x1)
display(dfcdx1)
dfcdx2 = deriv.(fc, x2)
display(dfcdx2)

result1 = evaluate(dfcdx1, x0) 
result2 = evaluate(dfcdx2, x0)
display(result1)
display(result2)

3-element Vector{Num}:
                    x2^2
 3x2*cos(x1)*(sin(x1)^2)
                     8x1

3-element Vector{Num}:
    2x1*x2
 sin(x1)^3
 2(3 + x2)

3-element Vector{Float64}:
 0.0
 0.0
 0.0

3-element Vector{Float64}:
 0.0
 0.0
 6.0